# Global & local vertical scale estimation

## Summary

Process results of progressive hypsometry (PH) for a collection of regions and their ensemble of "supercatchments" to identify the global and local vertical scales of each. 

Each supercatchments PH data file provides a set of $h_\mathrm{outlet}$ and $h_\mathrm{bench}$ values in tab-separated form. This table of PHBs (progressive hypsometry benches) is converted into an vector of $\Delta{h} = h_\mathrm{outlet}-h_\mathrm{bench}$ values.

Kernel density estimation is used to compute a smooth frequency distribution (pdf) of $\Delta{h}$ assuming a Gaussian kernel with a Silverman bandwidth (narrowed by 75%).

The peaks of this distribution, which are the modes of the pdf, are located, and the largest two, which likely constitute the global and local vertical scales, are chosen.

The kde pdf is then modeled through least-squares curve-fitting using two Gaussians, whose means are initiated at these two modes respectively, and whose means, standard deviations and relative magnitude are permitted to vary.

The result is an estimate of the global and local scales and their spread (one standard deviation) for each supercatchment. These results are gathered into one summary table and exported as a text file.

Graphs are plotted to record the kde pdf, the Gaussian-modeled pdf, the raw kde pdf modes and the Gaussian modeled modes aka global and local vertical scales.

### Prerequisites

To run this notebook you’ll need to have installed (with `Conda`, `pip` or other package manager) the following packages:
  - numpy
  - scipy
  - matplotlib
  - os
  - pandas
  - sklearn
  - json

### How to run

1. Open a terminal
2. Assuming you are running `bash`, type: 
    - `export PHHOME="${HOME}/Science/ProgHypso"` 
    - or whatever your path to the parent for `PHscales` is
3. `cd` to the directory containing the notebook
4. Type `jupyter-notebook`, or to choose a specific version, e.g.  `jupyter-notebook-3.7`
5. `Cell` -> `Run all`

## Preliminaries

In [1]:
import json, os, pandas as pd, numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.neighbors.kde import KernelDensity
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
from scipy.stats import norm

In [2]:
ph_home_path=os.getenv('PHHOME',
                os.path.join(os.getenv('HOME'),'Science','ProgHypso'))
if ph_home_path is None:
    ph_home_path=os.path.join(os.getenv('HOME'),'Science','ProgHypso')
ph_scales_path = os.path.join(ph_home_path,'PHscales')

In [3]:
# figsize = [6, 8]
figsize = [8, 6]
mpl.rc( 'figure', autolayout=False,  titlesize='Large',dpi=100)
mpl.rc( 'lines', linewidth=2.0, markersize=8)
mpl.rc( 'font', size=12, family='Arial')
mpl.rc( 'axes', labelsize=12) 

In [4]:
bandwidth_adjustment_factor = 0.75
bandwidth_adjustment_factor
example_phb_index = 10
default_gaussian_stdev = 20

## Methods

In [5]:
def read_data(dir_name=('.'), file_name=None, 
              file_ext='', key='ph', sep='\t',
              index_col=0, header=0, skip_rows=None):
    """
    """
    dir_name = os.path.join(*dir_name)
    if not os.path.exists(dir_name):
        print('Cannot find data directory')
        raise
    try:
        file_path = os.path.join(dir_name,file_name+file_ext)
        df = pd.read_csv(file_path, sep=sep,
                           index_col=index_col, header=header, 
                           skiprows=skip_rows) 
    except OSError:  
        print('Cannot read data file {}'.format(file_path))
        raise
    except:  
        raise
    return df

In [6]:
def bimodal_model(x,x0,s0,a0,x1,s1):
    return ( a0*norm.pdf(x, loc=x0,scale=s0)
            +(1-a0)*norm.pdf(x, loc=x1,scale=s1) )

In [7]:
def trimodal_model(x,x0,s0,a0,x1,s1,a1,x2,s2):
    return ( a0*norm.pdf(x, loc=x0,scale=s0)
            +a1*norm.pdf(x, loc=x1,scale=s1)
            +(1-a0-a1)*norm.pdf(x, loc=x2,scale=s2) )

In [8]:
def compute_kde_pdf(phbs_dict, phb):
    raw_df = phbs_dict[phb]['raw_df']
    data = np.array(raw_df['Delta_h']).reshape(-1,1)
    Delta_h_array = np.linspace(0,1.2*np.max(data),200)
    bw = data.std()*(4/3/data.size)**(1/5)*bandwidth_adjustment_factor
    print('{}:  bandwidth = {:.0f}'.format(phb,bw))
    kde = KernelDensity(kernel='gaussian', bandwidth=bw)
    kde.fit(data)
    kde_array = np.exp(kde.score_samples(Delta_h_array.reshape(-1,1)))
    kde_df = pd.DataFrame(index=None,
                          data={'Delta_h': Delta_h_array,
                                'p_Delta_h': kde_array})
    phbs_dict[phb].update({'kde_df':kde_df})

In [9]:
def find_modes(phbs_dict, phb):
    kde_df = phbs_dict[phb]['kde_df']
    Delta_h_array = kde_df['Delta_h']
    p_Delta_h_array = kde_df['p_Delta_h']
    peaks_array = find_peaks(p_Delta_h_array)[0]
    peaks_array = np.vstack((peaks_array,
                             Delta_h_array[peaks_array],
                             p_Delta_h_array[peaks_array])).T
    peaks_array = np.flipud(peaks_array[peaks_array[:,2].argsort()])
    principal_modes_array = peaks_array[:,1:3]
    phbs_dict[phb].update({'modes':principal_modes_array})

    print(phb)
    print('Principal mode#1: ∆h={:.0f}m  p(∆h)={:.5f}'
          .format(principal_modes_array[0,0],principal_modes_array[0,1]))
    print('Principal mode#2: ∆h={:.0f}m  p(∆h)={:.5f}'
          .format(principal_modes_array[1,0],principal_modes_array[1,1]))
    if principal_modes_array.shape[0]>2:
        print('Principal mode#3: ∆h={:.0f}m  p(∆h)={:.5f}'
          .format(principal_modes_array[2,0],principal_modes_array[2,1]))
    print('')

In [10]:
def fit_bimodal(phbs_dict, phb):
    kde_df          = phbs_dict[phb]['kde_df']
    modes_array     = phbs_dict[phb]['modes']
    Delta_h_array   = kde_df['Delta_h']
    p_Delta_h_array = kde_df['p_Delta_h']
    initial_values  = [modes_array[0,0], default_gaussian_stdev, 0.5,
                       modes_array[1,0], default_gaussian_stdev]
    popt,pcov = curve_fit(bimodal_model, 
                          Delta_h_array, p_Delta_h_array, 
                          p0=initial_values)
    phbs_dict[phb].update({'bimodal':(popt,pcov)})
    
    print(phb)
#     print(modes_array)
    print('Global vertical scale: H_G = {:.0f} ± {:.0f}m'
          .format(popt[0],popt[1]))
    print('Local vertical scale:  H_L = {:.0f} ± {:.0f}m'
          .format(popt[3],popt[4]))
    print()

In [11]:
def fit_trimodal(phbs_dict, phb):
    kde_df          = phbs_dict[phb]['kde_df']
    modes_array     = phbs_dict[phb]['modes']
    Delta_h_array   = kde_df['Delta_h']
    p_Delta_h_array = kde_df['p_Delta_h']
    if modes_array.shape[0]>2:
        initial_values  = [modes_array[0,0], default_gaussian_stdev, 0.5,
                           modes_array[1,0], default_gaussian_stdev, 0.4,
                           modes_array[2,0], default_gaussian_stdev]
        popt,pcov = curve_fit(trimodal_model, 
                              Delta_h_array, p_Delta_h_array, 
                              p0=initial_values)
        phbs_dict[phb].update({'trimodal':(popt,pcov)})
        print(phb)
#         print(modes_array)
        print('Global vertical scale: H_G = {:.0f} ± {:.0f}m'
              .format(popt[0],popt[1]))
        print('Local vertical scale:  H_L = {:.0f} ± {:.0f}m'
              .format(popt[3],popt[4]))
        print('Extra vertical scale:  H_X = {:.0f} ± {:.0f}m'
              .format(popt[6],popt[7]))
        print()
    else:
        print(phb)
        print('Two modes only resolved')
        print()
        phbs_dict[phb].update({'trimodal':(None,None)})

In [12]:
# Defunct - for archiving and possible re-use later
def gmm():
    nc_list = [2]
    gmm_list = [GaussianMixture(n_components=nc,
                means_init=peaks_array[0:2,1].reshape(2,1)) 
                for nc in nc_list]
    model_list = [gmm.fit(data) for gmm in gmm_list]
    model_aic_list = [model.aic(data) for model in model_list]
    best_nc = nc_list[np.argmin(model_aic_list)]
    best_model = model_list[np.argmin(model_aic_list)]
    best_model_pdf = np.exp(
        best_model.score_samples(Delta_h_array.reshape(-1,1)))
    best_model_mean= best_model.means_[np.argmax(best_model.precisions_)]
    best_model_stdev \
        = np.sqrt(1.0/best_model.precisions_
                  [np.argmax(best_model.precisions_)]).ravel()
    best_results = np.vstack((np.ravel(best_model.means_),
                          np.sqrt(1/best_model.precisions_).ravel())).T
    best_results = best_results[best_results[:,1].argsort()]
    best_results
    best_results[0]


    nc = 2
    bgmm = BayesianGaussianMixture(n_components=nc,
                                   mean_prior=best_model_mean)
    bayes_model = bgmm.fit(data)
    bayes_model_pdf = np.exp(
        bayes_model.score_samples(Delta_h_array.reshape(-1,1)))
    bayes_results = np.vstack((np.ravel(bayes_model.means_),
                            np.sqrt(1/bayes_model.precisions_).ravel(),
                               np.ravel(bayes_model.weights_)
                              )).T
    bayes_results
    bayes_results = bayes_results[bayes_results[:,2].argsort()]




    model_ph_df = pd.DataFrame(index=None,
                data={'Delta_h': Delta_h_array, 
                      'GMM_pd': best_model_pdf, 
                      'BGMM_pd': bayes_model_pdf})
    model_ph_dict = {}
    model_ph_dict.update({data_set_name: model_ph_df})

In [13]:
def plot_pdf():
    figure = plt.figure(figsize=figsize)
    plt.plot(Delta_h_array,kde_array, 
             c='DarkGreen', lw=1,
             label='kde pdf')
    plt.plot(principal_modes_array[0,0],principal_modes_array[0,1],
             'o',ms=8,c='DarkGreen',fillstyle='none')
    plt.plot(principal_modes_array[1,0],principal_modes_array[1,1],
             's',ms=8,c='DarkGreen',fillstyle='none')
    plt.plot(Delta_h_array, bimodal_model(Delta_h_array, *popt),
             c='DarkBlue',lw=2,
             label='dual Gaussian fit')
    plt.plot(popt[0],bimodal_model(popt[0], *popt),
             'o',ms=6,c='DarkBlue',
             label=r'global scale  $H_G \approx ${}$\pm${}m'\
                 .format(int(np.round(popt[0])),int(np.round(popt[1])) ))
    plt.plot(popt[3],bimodal_model(popt[3], *popt),
             's',ms=6,c='DarkBlue',
             label=r' local scale   $H_L \approx ${}$\pm${}m'\
                 .format(int(np.round(popt[3])),int(np.round(popt[4])) ))
    axes = plt.gca()
    axes.set_ylim(0,)
    axes.set_xlim(0,1.2*np.max(ph_df['Delta_h']));
    if popt[0]>np.max(ph_df['Delta_h'])/2:
        loc='upper left'
    else:
        loc='upper right'
    plt.legend(fontsize=12,loc=loc)
    plt.title(data_set_name);

## Data

In [14]:
phbs_group = ['HalfSqKmAc','PHBs','Cusum02_BenchLength3Steps','Tables']

In [15]:
region_dict = {}
for dir in os.listdir(ph_scales_path):
    path = os.path.realpath(os.path.join(ph_scales_path,dir))
    if dir[0]!='.' and dir!='Analysis' and os.path.isdir(path):
        region_dict.update({dir : path})
region_dict 

{'CostaRica': '/Users/max/Science/ProgHypso/PHscales/CostaRica',
 'Finisterres': '/Users/max/Science/ProgHypso/PHscales/Finisterres',
 'GabilanMesa': '/Users/max/Science/ProgHypso/PHscales/GabilanMesa',
 'SanGabriel': '/Users/max/Science/ProgHypso/PHscales/SanGabriel',
 'SantaMarta': '/Users/max/Science/ProgHypso/PHscales/SantaMarta'}

In [16]:
supercatchment_dict = {}
for region in region_dict:
    path = os.path.join(region_dict[region],*phbs_group)
    supercatchment_list = [(path,supercatchment)
                            for supercatchment in os.listdir(path) 
                            if supercatchment[0]!='.']
    supercatchment_dict.update({region : supercatchment_list})
supercatchment_dict

FileNotFoundError: [Errno 2] No such file or directory: '/Users/max/Science/ProgHypso/PHscales/GabilanMesa/HalfSqKmAc/PHBs/Cusum02_BenchLength3Steps/Tables'

In [ ]:
phbs_dict = {}

In [ ]:
for supercatchment in supercatchment_dict.keys():
    for phbs in supercatchment_dict[supercatchment]:
        name = supercatchment+'_'+phbs[1].replace('.txt','')
        path = (phbs[0],phbs[1])
        df = read_data(dir_name=[phbs[0]], 
                          file_name=phbs[1], index_col=None)
        phbs_dict.update({name : {'path':phbs[0],
                                  'file':phbs[1],
                                  'raw_df':  df} })

list(phbs_dict.keys())

In [ ]:
example_phb = list(phbs_dict.keys())[example_phb_index]
df = phbs_dict[example_phb]['raw_df']
axes = df.plot.scatter(x='Outlets',y='Modes',title=example_phb)
axes.set_ylim(0,);

## Analysis

For every supercatchment result table (now in a `pandas` dataframe), compute $\Delta{h} = h_\mathrm{mode}-h_\mathrm{outlet}$ and put into a new column in that dataframe.

In [ ]:
for phb in phbs_dict:
    df = phbs_dict[phb]['raw_df']
    df['Delta_h'] = df['Modes']-df['Outlets']

Check the pdf $p(\Delta{h})$ for an example super.

In [ ]:
example_phb = list(phbs_dict.keys())[example_phb_index]

df = phbs_dict[example_phb]['raw_df']
axes = df.plot.density(x='Outlets',y='Delta_h',bw_method='silverman',
                          title=example_phb)
axes.set_xlim(0,1.1*np.max(df['Delta_h']));
axes.set_ylim(0,);

Compute a kernel-density estimated pdf for each super, and place in another dataframe for each.

In [ ]:
for phb in phbs_dict:
    compute_kde_pdf(phbs_dict, phb)

For the example super, compare this kde pdf with the one generated and plotted above by `pandas`.

In [ ]:
example_phb = list(phbs_dict.keys())[example_phb_index]
df = phbs_dict[example_phb]['kde_df']
axes = df.plot.line(x='Delta_h',y='p_Delta_h',
                          title=example_phb)
axes.set_xlim(0,np.max(df['Delta_h']));
axes.set_ylim(0,);

In [ ]:
for phb in phbs_dict:
    modes = find_modes(phbs_dict, phb)

In [ ]:
for phb in phbs_dict:
    fit_bimodal(phbs_dict, phb)

In [ ]:
for phb in phbs_dict:
    fit_trimodal(phbs_dict, phb)

## Plotting

## Export